### Builder
##### [Referencia 1](https://refactoring.guru/pt-br/design-patterns/builder)

#### O `Builder` é um padrão de projeto criacional que permite a você construir objetos complexos passo a passo. O padrão permite que você produza diferentes tipos e representações de um objeto usando o mesmo código de construção.
![img](https://refactoring.guru/images/patterns/content/builder/builder-pt-br.png)

#### Imagine um `objeto complexo` que necessite de uma inicialização passo a passo trabalhosa de muitos campos e objetos agrupados. Tal código de inicialização fica geralmente enterrado dentro de um `construtor monstruoso com vários parâmetros`. Ou pior: espalhado por todo o código cliente.
![img](https://refactoring.guru/images/patterns/diagrams/builder/problem1.png)

#### Por exemplo, vamos pensar sobre como criar um objeto `Casa`. Para construir uma casa simples, você precisa construir quatro paredes e um piso, instalar uma porta, encaixar um par de janelas, e construir um teto. Mas e se você quiser uma casa maior e mais iluminada, com um jardim e outras miudezas (como um sistema de aquecimento, encanamento, e fiação elétrica)?

#### `A solução mais simples é estender a classe base Casa e criar um conjunto de subclasses para cobrir todas as combinações de parâmetros`. Mas eventualmente você acabará com um número considerável de subclasses. Qualquer novo parâmetro, tal como o estilo do pórtico, irá forçá-lo a aumentar essa hierarquia cada vez mais.

#### Há outra abordagem que não envolve a propagação de subclasses. `Você pode criar um construtor gigante diretamente na classe Casa base com todos os possíveis parâmetros que controlam o objeto casa.` Embora essa abordagem realmente elimine a necessidade de subclasses, ela cria outro problema.
![img](https://refactoring.guru/images/patterns/diagrams/builder/problem2.png)

#### `Na maioria dos casos a maioria dos parâmetros não será usada, tornando as chamadas do construtor em algo feio de se ver`. Por exemplo, apenas algumas casas têm piscinas, então os parâmetros relacionados a piscinas serão inúteis nove em cada dez vezes.

### Solução

#### O padrão Builder sugere que você extraia o código de `construção do objeto para fora de sua própria classe e mova ele para objetos separados chamados builders`. “Builder” significa “construtor”, mas não usaremos essa palavra para evitar confusão com os construtores de classe.
![img](https://refactoring.guru/images/patterns/diagrams/builder/solution1.png)


#### `O padrão organiza a construção de objetos em uma série de etapas (construirParedes, construirPorta, etc.)`. Para criar um objeto você executa uma série de etapas em um objeto builder. A parte importante é que você não precisa chamar todas as etapas. Você chama apenas aquelas etapas que são necessárias para a produção de uma configuração específica de um objeto.

#### Algumas das etapas de `construção podem necessitar de implementações diferentes` quando você precisa construir várias representações do produto. Por exemplo, paredes de uma cabana podem ser construídas com madeira, mas paredes de um castelo devem ser construídas com pedra.

#### Nesse caso, `você pode criar diferentes classes construturas que implementam as mesmas etapas de construção`, mas de maneira diferente. Então você pode usar esses builders no processo de construção (i.e, um pedido ordenado de chamadas para as etapas de construção) para produzir diferentes tipos de objetos.
![img](https://refactoring.guru/images/patterns/content/builder/builder-comic-1-pt-br.png)

#### Por exemplo, imagine um builder que `constrói tudo` de madeira e vidro, um segundo builder que `constrói tudo` com pedra e ferro, e um terceiro que usa ouro e diamantes. Ao chamar o mesmo conjunto de etapas, você obtém uma casa normal do primeiro builder, um pequeno castelo do segundo, e um palácio do terceiro. Contudo, isso só vai funcionar se o código cliente que chama as etapas de construção é capaz de interagir com os builders usando uma interface comum.

### Diretor

#### Você pode ir além e extrair uma série de chamadas para as etapas do builder que você usa para construir um produto em uma `classe separada chamada diretor`. A classe diretor define a ordem na qual executar as etapas de construção, enquanto que o builder provê a implementação dessas etapas.
![img](https://refactoring.guru/images/patterns/content/builder/builder-comic-2-pt-br.png)


#### *Ter uma classe diretor em seu programa não é estritamente necessário*. Você sempre pode chamar as etapas de construção em uma ordem específica diretamente do código cliente. Contudo, a classe diretor pode ser um bom lugar para colocar várias rotinas de construção para que você possa reutilizá-las em qualquer lugar do seu programa.

#### Além disso, a classe diretor esconde completamente os detalhes da construção do produto do código cliente. O cliente só precisa associar um builder com um diretor, inicializar a construção com o diretor, e então obter o resultado do builder.

### Exemplo conceitual(Código)

In [33]:
from abc import abstractmethod, ABC
from typing import Any, Callable

class Builder(ABC):
    @abstractmethod
    def process(self) -> None:
        ...
    
    @abstractmethod
    def process_part_a(self) -> None:
        ...

    @abstractmethod
    def process_part_b(self) -> None:
        ...

    @abstractmethod
    def process_part_c(self) -> None:
        ...             


class HomeSimple:
    def __init__(self) -> None:
        self._struct = []
        
    def add_part(self, item: Any) -> None:
        self._struct.append(item)
    
    def list_items(self) -> list:
        return self._struct


class ConcretBuilderHome(Builder):
    def __init__(self) -> None:
        self.reset()
        
    def reset(self) -> None:
        self._product: HomeSimple = HomeSimple()

    def process_part_a(self) -> None:
        self._product.add_part('floor')

    def process_part_b(self) -> None:
        self._product.add_part('wall')

    def process_part_c(self) -> None:
        self._product.add_part('window')
        
    def process(self) -> None:
        self.process_part_a()
        self.process_part_b()
        self.process_part_c()


class Director:
    def __init__(self) -> None:
        self._builder: Any = None
        
    def builder(self, builder: Builder) -> None:
        self._builder = builder

    def build_minimal_viable_product(self) -> None:
        self._builder.process_part_a()

    def build_full_featured_product(self) -> None:
        self._builder.process_part_a()
        self._builder.process_part_b()
        self._builder.process_part_c()
        


standerhome = ConcretBuilderHome()
standerhome.process()
print(standerhome._product.list_items())

myhome = ConcretBuilderHome()
for _ in range(4):
    myhome.process_part_a()
    
for _ in range(5):
    myhome.process_part_b()
    myhome._product.add_part('roof')
    
myhome.process_part_c()
print(myhome._product.list_items())

director = Director()
director.builder(ConcretBuilderHome())
director.build_full_featured_product()
print(director._builder._product.list_items())


class ProductManager(ABC):
    def __init__(self, parts: Any, size: float, color: str) -> None:
        self.parts: Any = parts
        self.size: float = size
        self.color: str = color

    @classmethod
    def builder(cls, parts: Any, size: float, color: str) -> None:
        return cls(parts, size, color)

    @abstractmethod
    def process(self) -> None:
        ...


class ProductA(ProductManager):
    def process(self) -> None:
        print(f"process: {self.parts} {self.size} {self.color}")
        
a = ProductA('A', 2.0, 'red')
a.process()

b = ProductA.builder('N', 7.0, 'blue')
b.process()

['floor', 'wall', 'window']
['floor', 'floor', 'floor', 'floor', 'wall', 'roof', 'wall', 'roof', 'wall', 'roof', 'wall', 'roof', 'wall', 'roof', 'window']
['floor', 'wall', 'window']
process: A 2.0 red
process: N 7.0 blue
